In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import networkx as nx
import igraph as ig

init_notebook_mode(connected=True)

In [4]:
G=nx.random_geometric_graph(200,0.125)
pos=nx.get_node_attributes(G,'pos')

dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d

edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Viridis',
        reversescale=False,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_info = '# of connections: '+str(len(adjacencies[1]))
    node_trace['text']+=tuple([node_info])
	
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

iplot(fig)

In [16]:
# G=ig.Graph.GRG(200,0.125)
# layout = G.layout("kk")

def get_plotly_data(E, coords):
    # E is the list of tuples representing the graph edges
    # coords is the list of node coordinates assigned by igraph.Layout
    N = len(coords)
    Xnodes = [coords[k][0] for k in range(N)]  # x-coordinates of nodes
    Ynodes = [coords[k][1] for k in range(N)]  # y-coordnates of nodes

    Xedges = []
    Yedges = []
    for e in E:
        Xedges.extend([coords[e[0]][0], coords[e[1]][0], None])
        Yedges.extend([coords[e[0]][1], coords[e[1]][1], None])

    return Xnodes, Ynodes, Xedges, Yedges

def plotly_graph(kmgraph=None, graph_layout='kk', colorscale='Viridis',
                 reversescale=False, showscale=True, factor_size=2,
                 keep_kmtooltips=True,
                 edge_linecolor='rgb(200,200,200)', edge_linewidth=1.5):
    # kmgraph: a dict returned by the method visualize, when path_html=None
    # graph_layout: an igraph layout; recommended 'kk' or 'fr'
    # factor_size: a factor for the node size
    # keep_tooltip: True  to keep the tooltips assigned by kmapper;
    # False, when kmapper tooltips contains projection statistic

    # define an igraph.Graph instance of n_nodes
    G=ig.Graph.GRG(200,0.125)
    
    links = G.get_edgelist()
    layt = G.layout(graph_layout)

    color_vals = None
    # size = np.array([factor_size * node['size'] for node in kmgraph['nodes']],
                   # dtype=np.int)
    Xn, Yn, Xe, Ye = get_plotly_data(links, layt)
    edges_trace = dict(type='scatter',
                       x=Xe,
                       y=Ye,
                       mode='lines',
                       line=dict(color=edge_linecolor,
                                 width=edge_linewidth),
                       hoverinfo='none')

    nodes_trace = dict(type='scatter',
                       x=Xn,
                       y=Yn,
                       mode='markers',
                       marker=dict(symbol='dot',
                                   colorscale=colorscale,
                                   showscale=showscale,
                                   reversescale=reversescale,
                                   colorbar=dict(thickness=20,
                                                 ticklen=4)),
                       text=None,
                       hoverinfo='text')

    return [edges_trace, nodes_trace]


def plot_layout(title='Test', width=700, height=700,
                bgcolor='rgba(20,20,20, 0.8)', annotation_text=None,
                annotation_x=0, annotation_y=-0.07):
    # width, height: plot window width, height
    # bgcolor: plot background color
    # annotation_text: meta data to be displayed
    # annotation_x, annotation_y are the coordinates of the
    # point where we insert the annotation

    # set the axes style; No axes!
    axis = dict(showline=False,
                zeroline=False,
                showgrid=False,
                showticklabels=False,
                title='')

    pl_layout = dict(title=title,
                     font=dict(size=12),
                     showlegend=False,
                     autosize=False,
                     width=width,
                     height=height,
                     xaxis=dict(axis),
                     yaxis=dict(axis),
                     hovermode='closest',
                     plot_bgcolor=bgcolor)

    if annotation_text is None:
        return pl_layout
    else:
        annotations = [dict(showarrow=False,
                            text=annotation_text,
                            xref='paper',
                            yref='paper',
                            x=annotation_x,
                            y=annotation_y,
                            xanchor='left',
                            yanchor='top',
                            font=dict(size=14))]
        pl_layout.update(annotations=annotations)
        return pl_layout

plotly_graph_data = plotly_graph()
layout = plot_layout()

iplot(dict(data=plotly_graph_data, layout=layout))